# Feed Forward Neural Networks

## What we will accomplish

In this notebook we will:
- Introduce Feed Forward Neural Networks
- Demonstrate how to implement them using `keras`.

In [ ]:
## For data handling
import pandas as pd
import numpy as np

## For plotting
import matplotlib.pyplot as plt
from seaborn import set_style

## This sets the plot style
## to have a grid on a white background
set_style("whitegrid")

import pprint

## Feed forward network architecture

Recall our parametric supervised learning framework: 
* We have *data* of the form $D = {(x_i, y_i)}$ with $x_i \in \mathbb{R}^p$ and $y_i \in \mathbb{R}^m$. 
* We have a collection of *hypothesized relationships* $f_\theta : \mathbb{R}^p \to \mathbb{R}^m$ depending on some *parameters* $\theta \in \Theta$.
* We have a *loss function* $\ell$ which takes as input a dataset $D$ and hypothesized function $f_\theta$ and returns a positive real number.
* We *fit* a model by finding the parameters which minimize the loss (at least approximately, if not exactly).

Feed Forward Neural Networks fit into this framework.  Our collection of hypothesized relationships have the following special form:

$$
f_\theta: \mathbb{R}^p \stackrel{A_1}{\longrightarrow} \mathbb{R}^{h_1} \stackrel{\sigma_1}{\longrightarrow} \mathbb{R}^{h_1} \stackrel{A_2}{\longrightarrow} \mathbb{R}^{h_2} \stackrel{\sigma_2} {\longrightarrow} \mathbb{R}^{h_2} \stackrel{A_3}{\longrightarrow} \dots \mathbb{R}^{h_k} \stackrel{A_k}{\longrightarrow} \mathbb{R}^{h_k} \stackrel{\sigma_k} {\longrightarrow} \mathbb{R}^{m}
$$

Here each $A_i$ is an affine function of the form $A_i(\vec{x}) = W_i\vec{x} + b_i$ and each $\sigma_i$ is a non-linear function.  The most common choice for $\sigma_i$ is $\operatorname{ReLU}(x) = \max(0,x)$ (which is applied to each coordinate independently).

Note that if we *do* take all $\sigma_i = \operatorname{ReLU}$, then $f_\theta$ is a piecewise linear function.  This gives us an extremely flexible hypothesis class:  in fact one can prove that just taking functions of the form $f: \mathbb{R}^p \to \mathbb{R}^m$ of the form $f =  A_2 \circ \operatorname{ReLU} \circ A_1$ is sufficiently flexible to allow us to uniformly approximate **any** continuous function on a compact domain (given a large enough intermediate dimension $h$).  Results of this nature go by the name of [universal approximation theorems](https://en.wikipedia.org/wiki/Universal_approximation_theorem).

It is common to visualize neural networks with pictures like the following:

<img src="lecture_12_assets/network1.png" width="60%"></img>

The way to interpret this is that each circle represents a coordinate, each arrow represents an entry of a matrix $W_i$, and the activation functions and biases are omitted from the picture.

For instance the function $f:\mathbb{R}^3 \to \mathbb{R}^2$ given by $f(\vec{x}) = \operatorname{ReLU}(W\vec{x})$ with $W = \begin{bmatrix} 1 & 2 & 3 \\ 4 & 5 & 6\end{bmatrix}$ could be visualized with the following picture:

<img src="lecture_12_assets/network2.png" width="30%"></img>

$$
f_\theta: \mathbb{R}^p \stackrel{A_1}{\longrightarrow} \mathbb{R}^{h_1} \stackrel{\sigma_1}{\longrightarrow} \mathbb{R}^{h_1} \stackrel{A_2}{\longrightarrow} \mathbb{R}^{h_2} \stackrel{\sigma_2} {\longrightarrow} \mathbb{R}^{h_2} \stackrel{A_3}{\longrightarrow} \dots \mathbb{R}^{h_k} \stackrel{A_k}{\longrightarrow} \mathbb{R}^{h_k} \stackrel{\sigma_k} {\longrightarrow} \mathbb{R}^{m}
$$

Some neural network specific terminology:

Each $\mathbb{R}^{d}$ in the chain is called a **layer**.  Sometimes the map "$\sigma_i \circ A_i$" is called the "layer" instead.  
* The first $\mathbb{R}^p$ is called the **input layer**.
* The last $\mathbb{R}^m$ is called the **output layer**.
* Intermediate $\mathbb{R}^{h_i}$ are called **hidden layers**.
* Each coordinate of each $\mathbb{R}^d$ is called a **neuron**.
* The $\sigma_i$ are called **activation functions**.  
    * The analogy is to a neuron firing or not depending on the strength of a stimulus.
* Each individual entry of the matrix $W_i$ is called a **weight**.
* Each individual entry of the vector $b_i$ is called a **bias**.
* Taken together the weights and biases are the **parameters** of the model.
* The full specification of allowable $f_\theta$ is called the **model architecture**.
    * For now this just looks like specifying the dimensions and activation functions, but we will soon see that we can place additional restrictions on the $A_i$
    * One example is specifying that the weight matrix $W_i$ must have a particular constrained form.
    * If there are no special constraints on $A_i$ it is called a **linear layer**, **fully connected layer**, or **dense layer**.
* Using a numerical algorithm to adjust the parameters to decrease the loss function is called **training** the neural network.
    * We will most frequently use some variant of gradient descent.
* A trained model (i.e. an architecture together with specific values for the weights and biases) is called a **checkpoint**.

Note that a fully connected feed forward neural network will have $(p+1)h_1 + (h_1 +1)h2 + \dots + (h_i + 1)h_{i+1} + \dots + (h_k + 1)m$ parameters!  This can really add up quickly.

## A naive implementation

Let's implement our own version of a feed forward ReLU network to better understand the structure.  

We only need to specify a list of dimensions for each layer and (optionally) a final activation function.

In [ ]:
class ReLUNN():
    '''A FFNN whose layers have dimensions [p,h_1,h_2,..., h_k, m] and uses RELU activation in all but the final layer'''
    def __init__(self,dims,sigma = None):
        self.dims = dims
        self.l = len(dims) - 1 # number of functions between layers
        self.weights = [np.random.randn(dims[i+1],dims[i]) for i in range(self.l)] # randomly initialize weights
        self.biases = [np.random.randn(dims[i+1], 1) for i in range(self.l)] # making bias shape (-1,1) so addition with a matrix broadcasts correctly.
        self.sigma = sigma
    
    @staticmethod
    def _relu(t):
        '''Takes input numpy array and sets negative entries to 0.0'''
        t[t<0] = 0.0
        return t
    
    def predict(self, X):
        '''Takes matrix of features X as input, applies layers, then applies final activation function sigma'''
        X = X.astype(float)
        X = X.transpose() # rows of design matrix are inputs of predict
        for i in range(self.l):
            X = np.dot(self.weights[i], X) + self.biases[i]
            if i != self.l-1:
                X = self._relu(X)
        if self.sigma:
            X = self.sigma(X)
        return X.transpose() # rows of output matrix are the predictions

In [ ]:
nn = ReLUNN(dims = [2,3,1], sigma= None)
pprint.pprint(nn.__dict__)
X = np.array([[.1,.2],[.3,.4],[.5,.6]])
print(f"f(X) = {nn.predict(X)}")

The weights and biases would be adjusted during training to minimize the loss function.  After this is done we would predict as above!

## We have already seen some Neural Networks!

Several of the machine learning techniques we have covered can be seen as simple neural networks.  For example:

* Linear Regression
    * We have a set of hypothesized relationships of the form $\mathbb{R^p} \stackrel{A_1}{\longrightarrow} \mathbb{R}$.
    * Our loss function is Mean Squared Error.
    * In neural network terminology we are training a single layer network with no activation function.
* Logistic Regression
    * We have a set of hypothesizes relationships of the form $\mathbb{R^p} \stackrel{A_1}{\longrightarrow} \mathbb{R} \stackrel{\sigma}{\longrightarrow} \mathbb{R}$ where $\sigma$ is the sigmoid activation function.
    * Our loss function is binary cross-entropy.
* Multiclass Logistic Regression
    * We have a set of hypothesizes relationships of the form $\mathbb{R^p} \stackrel{A_1}{\longrightarrow} \mathbb{R^m} \stackrel{\operatorname{SoftMax}}{\longrightarrow} \mathbb{R}^m$ where $\operatorname{SoftMax}(\vec{a})$ has $i^{th}$ coordinate $\frac{\exp(a_i)}{\sum \exp(a_j)}$.
    * We use categorical cross-entropy as our loss function.


### Training a Neural Network:  Gradient Descent

In many of the machine learning algorithms we have covered so far, minimizing the loss function is relatively simple:

* In linear regression we have an explicit solution of the parameters which minimize the MSE.
* In logistic regression and SVM we don't have an explicit solutions, but we *do* have simple explicit formulas for the gradient and Hessian of the loss function which makes numerical optimization very fast.
* Many of the other techniques we have covered also have convex loss functions and/or have optimization algorithms which make training relatively quick and inexpensive.

Neural Networks are different.  The loss function, thought of as a function of the parameters, is often **highly** non-convex.  You should expect tons of saddle points, non-optimal local minima, and other terrible obstructions to numerical optimization.  Despite this (and somewhat mysteriously I might add!) neural networks trained using good old fashioned Gradient Descent are capable of amazing feats like writing poetry, identifying what song is playing, and driving cars!  Here is a brief summary of how Gradient Descent works:

* Choose a **learning rate** $\textrm{lr}$.
* Initialize your parameters $\theta$ randomly as $\theta_0$.
* At each stage $j$:
    * Compute the gradient of the loss function $\nabla \ell \big|_{\theta_j}$.
    * Set $\theta_{j+1} = \theta_j - \textrm{lr} \nabla \ell \big|_{\theta_j}$
* Stop when you reach some stopping condition (often when validation loss stops decreasing).

In the following picture we are showing the contour lines of a loss function of two parameters.  The gradient points in the direction of steepest ascent ("straight up the hill") while the negative gradient points in the direction of steepest descent ("straight down the hill").  The gradient descent algorithm just has us start at a random point and take one little step at a time in the direction which carries us down the fastest. This will generally lead us to a local (but not necessarily global) minimum of the loss function.

<img src="lecture_12_assets/contour.png" width="40%"></img>

The gradient is computed using the chain rule, which in this context is known as **backpropagation**.  Technically backpropagation is not just the chain rule, but is a specific numerical implementation of the chain rule which avoids the inefficiencies of a naive implementation.  You can think of it as roughly [chain rule](https://en.wikipedia.org/wiki/Chain_rule) + [memoization](https://en.wikipedia.org/wiki/Memoization) = [backpropagation](https://en.wikipedia.org/wiki/Backpropagation).  Note:  a previous Erdős Alumni said this to me (Steven Gubkin) in conversation and I believe they may have coined the phrase.  I do not recall who that was though!

#### Common adjustments to the gradient descent

Common adjustments to the gradient descent step:

1. Calculating the gradient of your loss function using a subset of your data rather than all of the data.  This is called **batch gradient descent**.  In the case that your batch size is one this is called **stochastic gradient descent**.  Getting through all of your data is called one **epoch**.

2. Adjusting your learning rate during training (aka adopting a **learning rate schedule**).  There are numerous methods.

3. [This wikipedia page](https://en.wikipedia.org/wiki/Stochastic_gradient_descent) contains an exposition of many different varients.

## Training simple feed forward neural networks with Keras

In [ ]:
import tensorflow as tf
import keras
from keras import layers
from tensorflow import convert_to_tensor

### A synthetic regression example

We will show how to approximate $\sin$ on the interval $[1,4]$ using a function of the form

$$
[1,4] \stackrel{A_1}{\longrightarrow} \mathbb{R}^9 \stackrel{\operatorname{ReLU}}{\longrightarrow} \stackrel{A_2}{\longrightarrow} \mathbb{R}^1
$$

by using a version of gradient descent called "RMSProp" to minimize the mean squared error loss between model outputs and true outputs.

In [ ]:
X = convert_to_tensor(np.linspace(1, 4, 1000))
y = convert_to_tensor(np.sin(X)+ 0.05*np.random.randn(1000))

# Create a Sequential model.  Specify input layer of dimension 1, BatchNormalization(axis = -1), a Dense layer of dimension 9, and a Dense layer of dimension 1.
model = 

# Compile model using "rmsprop" as the optimizer, and "mse" as the loss.

# Fit the model for 100 epochs.


In [ ]:
preds = model.predict(X).reshape(-1)

plt.scatter(X,y, alpha = 0.6)
plt.plot(X, preds, 'k')
# plotting points where the slope changes
for i in range(1,len(preds)-1):
    if np.abs((preds[i] - preds[i-1]) - (preds[i+1] - preds[i])) > 0.00001:
        plt.plot(X[i], preds[i], 'ko')
plt.show()

Note that our approximation is piecewise linear! We can extract the weights and biases as follows:

In [ ]:
model.weights

Can we use just these weights and biases to recover the picture we drew above?  

We could, but the batch normalization layer makes things a bit more complicated. 

Let's rerun it with the first two lines line commented out.  Note that I specified the input shape in the first Dense layer this time.

In [ ]:
model = keras.Sequential([
    #layers.InputLayer(1),  
    #layers.BatchNormalization(axis = -1), # equivalent of StandardScaler().  Puts inputs on "same scale" as initial parameters, which helps with training.
    layers.Dense(9, activation = 'relu', input_shape=(1,)), # R^1 --> R^9 with ReLU activation
    layers.Dense(1) # back down to R^1 again with a pure linear layer
])

model.compile(optimizer = "rmsprop", # version of gradient descent which adjusts learning rate dynamically
              loss = "mse", # Mean Squared Error loss function
              metrics = ["mean_squared_error"])

model.fit(X, y, epochs = 200) # we do 200 steps of the gradient descent algorithm

In [ ]:
# Putting weights in the form of our custom class

W_1 = model.weights[0].numpy().transpose()
b_1 = model.weights[1].numpy().reshape(-1,1)
W_2 = model.weights[2].numpy().transpose()
b_2 = model.weights[3].numpy().reshape(-1,1)

In [ ]:
W_1.shape, b_1.shape, W_2.shape, b_2.shape

In [ ]:
def relu(x):
    x[x<0] = 0
    return x

In [ ]:
inputs = X.numpy().reshape(1,-1)
y_np = b_2 + np.dot(
    W_2,
    relu(
        np.dot(W_1, inputs) + b_1
        )
    )

In [ ]:
plt.scatter(X,y, alpha = 0.6)
plt.plot(X, y_np.reshape(-1), "k")
plt.show()

Alternatively, we can make sure that our `RelUNN` class functions properly by plugging these learned weights in.

In [ ]:
nn = ReLUNN(dims = [1,9,1])
nn.weights = [W_1,W_2]
nn.biases = [b_1, b_2]
nn_preds = nn.predict(X.numpy().reshape(-1,1))

In [ ]:
plt.scatter(X,y, alpha = 0.6)
plt.plot(X, nn_preds, "k")
plt.show()

### A regression example with real data.

We use the same concrete data from problem set 11.

Yeh,I-Cheng. (2007). Concrete Compressive Strength. UCI Machine Learning Repository. https://doi.org/10.24432/C5PK67.

In [ ]:
concrete = pd.read_csv("../../data/concrete.csv") 
concrete

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
features = concrete.columns[:-1]
target = concrete.columns[-1]

X_train, X_test, y_train, y_test = train_test_split(concrete[features],concrete[target], test_size=0.2, random_state=216)

X_tt,X_val, y_tt, y_val = train_test_split(X_train,y_train, test_size=0.2, random_state=216)

X_train_tensor = convert_to_tensor(X_train)
y_train_tensor = convert_to_tensor(y_train)
X_tt_tensor = convert_to_tensor(X_tt)
X_val_tensor = convert_to_tensor(X_val)
X_test_tensor = convert_to_tensor(X_test)
y_tt_tensor = convert_to_tensor(y_tt)
y_val_tensor = convert_to_tensor(y_val)
y_test_tensor = convert_to_tensor(y_test)

In [ ]:
# Showcasing a learning rate scheduler
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    0.01, #initial learning rate
    decay_steps= 1000 , # After 1000 steps we start to decrease the learning rate
    decay_rate=0.95, # multiply learning rate by 0.95 for each epoch after
    staircase=True)

model = keras.Sequential([
    layers.InputLayer(8),
    layers.BatchNormalization(axis = -1),
    layers.Dense(20, activation = 'relu'),
    layers.Dense(20, activation = 'relu'),
    layers.Dense(1,kernel_regularizer=keras.regularizers.L2(0.01)) # Showing how to regularize weights
])

model.compile(keras.optimizers.legacy.Adam(learning_rate=lr_schedule), loss = "mse", metrics= [keras.metrics.RootMeanSquaredError()])

history = model.fit(X_tt_tensor, y_tt_tensor, epochs = 200, validation_data = (X_val_tensor, y_val_tensor), batch_size = 32) # we do 200 steps of the gradient descent algorithm

This is a really simple fully connected NN with two hidden layers. It gives us pretty similar performance to RandomForestRegressor without doing any hyperparameter tuning.

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
rfr.fit(X_tt,y_tt)
np.sqrt(mean_squared_error(rfr.predict(X_val),y_val))

### A classificiation example

In [ ]:
t = np.linspace(0,4*np.pi, 500)
X_0 = np.stack([t*np.cos(t), t*np.sin(t)], axis = 0).transpose() + 0.5*np.random.standard_normal((500,2))
X_1 = np.stack([(t+1.2)*np.cos(t), (t+1.2)*np.sin(t)], axis = 0).transpose() + 0.5*np.random.standard_normal((500,2))
X = np.concatenate([X_0,X_1], axis = 0)
y = np.concatenate([np.zeros(500), np.ones(500)])

In [ ]:
y.shape

In [ ]:
plt.scatter(X_0[:,0],X_0[:,1])
plt.scatter(X_1[:,0],X_1[:,1])

plt.show()

In [ ]:
clf = keras.Sequential([
    layers.Normalization(axis=-1), # Basically StandardScaler()
    layers.Dense(50, activation = 'relu', use_bias = True ,input_shape=(2,)), # R^2 --> R^12 with ReLU activation
    layers.Dense(50, activation = 'relu', use_bias = True), # R^12 --> R^12 with ReLU activation
    layers.Dense(1, use_bias = True, activation = 'sigmoid') # R^12 --> R^1 again with sigmoid activation
])

clf.compile(optimizer = "rmsprop", 
              loss = "binary_crossentropy", 
              metrics = ["accuracy"])

clf.fit(X, y, epochs = 500) 

In [ ]:
xx, yy = np.mgrid[-12:16:.1, -15:12:.1]
grid = np.c_[xx.ravel(), yy.ravel()]
probs = clf.predict(grid).reshape(xx.shape)

In [ ]:
contour = plt.contourf(xx, yy, probs, 25, alpha = 0.6,
                      vmin=0, vmax=1)
ax_c = plt.colorbar(contour)
ax_c.set_label("$P(y = 1)$")
ax_c.set_ticks([0, .25, .5, .75, 1])

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

plt.scatter(X[:,0],X[:,1],  c = [colors[int(j)] for j in y], alpha = 0.3)

plt.show()

--------------------------

This notebook was written for the Erdős Institute Data Science Boot Camp by Steven Gubkin.

Please refer to the license in this repo for information on redistribution.